In [1]:
## Following along work in:
# https://github.com/codingforentrepreneurs/30-Days-of-Python/blob/master/tutorial-reference/Day%2019/notebooks/4%20-%20Resource-Enabled%20Client.ipynb
# https://www.youtube.com/watch?v=xdq6Gz33khQ&ab_channel=CodingEntrepreneurs

In [ ]:
# This worked...
# pip install squarify

In [ ]:
# These didn't...
# pip install plotly_express==0.4.0
# !pip install plotly==4.14.1

In [ ]:
# # These ran endlessly, never timed out, didn't install
# import sys
# !{sys.executable} -m pip install plotly==4.14.1

In [ ]:
# import plotly
# pip freeze

In [ ]:
# Import libraries
import base64
import requests
import datetime
from urllib.parse import urlencode
import pandas as pd

In [ ]:
# Import graphing libraries
import matplotlib.pyplot as plt
import squarify # pip install squarify

In [ ]:
# Credentials, having registered your app with Spotify 
client_id = '9c9266182f5f404483f26ba070ed6e67'
client_secret = '0b6abef0cd414ecab0f76088a111c9a6'

In [ ]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        '''
        Returns a base64 encoded string (not bytes)
        '''
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception('You must set client_id and client_secret')
        client_creds = '{}:{}'.format(client_id, client_secret)
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            'Authorization':'Basic {}'.format(client_creds_b64)
        }
    
    def get_token_data(self):
        return {
            'grant_type':'client_credentials'
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)        
        if r.status_code not in range(200, 299):
            raise Exception('Could not authenticate client')
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_headers(self):
        access_token = self.get_access_token()
        headers = {
            'Authorization': 'Bearer {}'.format(access_token)
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = 'https://api.spotify.com/{}/{}/{}'.format(version, resource_type, lookup_id)
        headers = self.get_resource_headers()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
        
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type='artist'): # type
        headers = self.get_resource_headers()
        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({'q': query, 'type': search_type.lower()})
        lookup_url = '{}?{}'.format(endpoint, data)
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    

In [ ]:
spotify = SpotifyAPI(client_id, client_secret)

In [ ]:
# query_string = 'A lannister always pays his debts'
query_string = 'happy birthday baby'

In [ ]:
# Run search query
search_result = spotify.search(query_string, search_type='track')

In [ ]:
artist_dict = {}
for i in range(0, len(search_result['tracks']['items'])):
    
    # Retrieve artist name and id
    artist_name = search_result['tracks']['items'][i]['artists'][0]['name']
    artist_id = search_result['tracks']['items'][i]['artists'][0]['id']
    
    # Run function to get followers, genres, etc.
    artist_data = spotify.get_artist(artist_id)
#     artist_image = artist_data['images']
    artist_genres = artist_data['genres']
    artist_url = artist_data['external_urls']['spotify']
    artist_followers = artist_data['followers']['total']
    
    # Store in dict
    artist_dict[artist_name] = {
        'ID':artist_id,
        'Genres':artist_genres,
        'URL':artist_url,
        'Followers':artist_followers}
    
#     followers = search_result['tracks']['items'][i]['artists'][0]['followers']['total']
#     search_result_artists[artist_name] = artist_id
#     search_result_artists[artist_name] = {
#         'ID':artist_id,
#         'Followers':followers}

In [ ]:
# Build list of genres, extracting artist-level genre lists
unique_genres = []
for artist in artist_dict.keys():
    for genre in artist_dict[artist]['Genres']:
        if genre not in unique_genres:
            unique_genres.append(genre)
unique_genres.sort(reverse=False)

# Assign new keys for each unique genre
for genre in unique_genres:
    for artist in artist_dict.keys():
        if genre in artist_dict[artist]['Genres']:
            artist_dict[artist][genre.title()] = genre.title()

In [ ]:
unique_genres[0:5]

In [ ]:
# First genre listed for each artist
for artist in artist_dict.keys():
    if artist_dict[artist]['Genres'] == []:
        artist_dict[artist]['Primary Genre'] = None
    else:
        artist_dict[artist]['Primary Genre'] = artist_dict[artist]['Genres'][0]

In [ ]:
artist_primary_genres_df = pd.DataFrame.from_dict(
    artist_dict, orient='index')[['Primary Genre', 'Followers']]
artist_primary_genres_df.dropna(
    subset=['Primary Genre'], inplace=True)
artist_primary_genres_df.sort_values(['Followers'],
                                     ascending=False,
                                     inplace=True)

In [ ]:
artist_primary_genres_df.iloc[0:5]

In [ ]:
# DataFrame showing only unique genre Series
artist_genres_df = pd.DataFrame.from_dict(
    artist_dict, orient='index').drop(
    ['ID', 'Genres', 'URL', 'Followers'], axis=1)

In [ ]:
artist_followers_df = pd.DataFrame.from_dict(
    artist_dict, orient='index')[['Followers']]
artist_followers_df.sort_values(['Followers'],
                                ascending=False,
                                inplace=True)

In [ ]:
artist_followers_df.iloc[0:5]

In [ ]:
# Treemap of artists by follower count
squarify.plot(sizes=artist_followers_df['Followers'],
              label=artist_followers_df[0:5].index,
              alpha=0.8)
plt.axis('off')
plt.show()

In [ ]:
import plotly.express as px

In [ ]:

fig = px.treemap(df, 
                 path=['Platform', 'Genre'], 
                 values='Global_Sales',
                 color='NA_Sales'
                )
fig.show()

In [ ]:
# Treemap of artists by genre by follower count
squarify.plot(sizes=artist_followers_df['Followers'],
              label=artist_followers_df[0:5].index,
              alpha=0.8)
plt.axis('off')
plt.show()

In [ ]:
# https://towardsdatascience.com/treemap-basics-with-python-777e5ed173d0

import matplotlib.pyplot as plt
import squarify # pip install squarify
import pandas as pd

sizes = [50, 25, 12, 6]
squarify.plot(sizes)
plt.show()

sizes=[50, 25, 12, 6]
label=["50", "25", "12", "6"]
squarify.plot(sizes=sizes, label=label, alpha=0.6 )
plt.axis('off')
plt.show()

sizes=[50, 25, 12, 6]
label=["50", "25", "12", "6"]
color=['red','blue','green','grey']
squarify.plot(sizes=sizes, label=label, color=color, alpha=0.6 )
plt.axis('off')
plt.show()

sizes=[50, 25, 12, 6]
label=["BC 1", "OT 1", "OT 2", "OT 3"]
color=['red','#1C9FB0','#32A0CE','#1C51B0']
squarify.plot(sizes=sizes, label=label, color=color, alpha=0.6 )
plt.axis('off')
plt.show()


In [ ]:
artist_dict['Hiss Golden Messenger']